# Zig-zagging across a graph

- https://adventofcode.com/2023/day/8

This puzzle presents us with a graph and a zig-zagging path across the nodes. I'll just implement the graph walking algorithm for now, even though I suspect that step two will up the ante somehow (probably involving loops or a modulus of some sort).


In [1]:
import typing as t
from dataclasses import dataclass
from itertools import cycle

Step: t.TypeAlias = t.Literal["L", "R"]


class DesertNode(t.NamedTuple):
    name: str
    left: str
    right: str


@dataclass
class DesertMap:
    nodes: dict[str, DesertNode]

    @classmethod
    def from_lines(cls, lines: t.Sequence[str]) -> t.Self:
        nodes: dict[str, DesertNode] = {}
        for line in lines:
            name, _, rem = line.partition("=")
            left, _, right = rem.strip(" ()").partition(",")
            nodes[name.strip()] = DesertNode(name.strip(), left.strip(), right.strip())
        return cls(nodes)

    def traverse(self, from_: DesertNode, step: Step) -> DesertNode:
        match step:
            case "L":
                return self.nodes[from_.left]
            case "R":
                return self.nodes[from_.right]

    def from_aaa_to_zzz(self, steps: t.Iterable[Step]) -> int:
        node = self.nodes["AAA"]
        for s, step in enumerate(cycle(steps), 1):
            if (node := self.traverse(node, step)).name == "ZZZ":
                return s
        raise AssertionError("Goal not reached")


def parse_steps(stepline: str) -> list[Step]:
    return [s for s in stepline if s in ("L", "R")]


tests: dict[str, int] = {
    """RL

AAA = (BBB, CCC)
BBB = (DDD, EEE)
CCC = (ZZZ, GGG)
DDD = (DDD, DDD)
EEE = (EEE, EEE)
GGG = (GGG, GGG)
ZZZ = (ZZZ, ZZZ)
""": 2,
    """LLR

AAA = (BBB, BBB)
BBB = (AAA, ZZZ)
ZZZ = (ZZZ, ZZZ)
""": 6,
}
for test, expected in tests.items():
    stepline, _, *maplines = test.splitlines()
    steps = parse_steps(stepline)
    assert DesertMap.from_lines(maplines).from_aaa_to_zzz(steps) == expected

In [2]:
import aocd

stepline, _, *maplines = aocd.get_data(day=8, year=2023).splitlines()
steps = parse_steps(stepline)
desert_map = DesertMap.from_lines(maplines)
print("Part 1:", desert_map.from_aaa_to_zzz(steps))

Part 1: 21389


# Follow in the steps of ghosts

Indeed, step two ups the ante by making you follow N paths at the same time. And those paths loop at different intervals, meaning they reach a `Z` point regularily but at different 'distances'. Track those distances, and once you have a loop for all different paths you can calculate their [least common multiple](https://en.wikipedia.org/wiki/Least_common_multiple).

(We've seen a similar puzzle before, [day 12 of 2019](../2019/Day%2012.ipynb)).


In [3]:
from math import lcm


class GhostlyDesertMap(DesertMap):
    @property
    def starting_points(self) -> tuple[DesertMap, ...]:
        return tuple(node for node in self.nodes.values() if node.name[-1] == "A")

    def from_as_to_zs(self, steps: t.Iterable[Step]) -> int:
        nodes = self.starting_points
        loops: list[int | None] = [None] * len(nodes)
        for s, step in enumerate(cycle(steps), 1):
            nodes = tuple(self.traverse(n, step) for n in nodes)
            for i, node in enumerate(nodes):
                if not loops[i] and node.name[-1] == "Z":
                    loops[i] = s
            if all(loops):
                return lcm(*loops)

        raise AssertionError("Goal never reached")


test = """LR

11A = (11B, XXX)
11B = (XXX, 11Z)
11Z = (11B, XXX)
22A = (22B, XXX)
22B = (22C, 22C)
22C = (22Z, 22Z)
22Z = (22B, 22B)
XXX = (XXX, XXX)
"""

test_stepline, _, *test_maplines = test.splitlines()
test_map = GhostlyDesertMap.from_lines(test_maplines)
assert test_map.from_as_to_zs(parse_steps(test_stepline)) == 6

In [4]:
map = GhostlyDesertMap.from_lines(maplines)
print("Part 2:", map.from_as_to_zs(steps))

Part 2: 21083806112641
